# Part 1: Ingestion and Basic RAG

Welcome to the **Deepnote RAG Training** series. In this first notebook, we will build the foundation of a Retrieval-Augmented Generation (RAG) system.

## What is RAG?
Large Language Models (LLMs) like GPT-4 are frozen in time. They don't know about your private data, your company's contracts, or the latest financial report from yesterday. **RAG** solves this by:
1. **Retrieving** relevant documents based on your query.
2. **Augmenting** the prompt with these documents.
3. **Generating** an answer using the LLM + the context.

## 1. Setup
First, we load the environment variables (API Keys) and import necessary libraries.

In [2]:
%pip install -r ../requirements.txt
import os
import openai
from dotenv import load_dotenv

load_dotenv()

# If using Deepnote's integration, the key is already in env
# os.environ["OPENAI_API_KEY"] = "sk-..."

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


True

## 2. Document Loading
**Concept:** Your data exists in many formats (PDF, Markdown, CSV, JSON). we need to standardise it into a `Document` object (text + metadata).

We will load the synthetic data we created in the `data/` folder.

### 📘 Poznámky k načítání dat (Data Ingestion)

Tento krok slouží k **normalizaci dat**. Nezáleží na tom, zda je zdrojovým souborem [.txt](cci:7://file:///c:/Users/jan.petr/OneDrive%20-%20dolphinconsulting.cz/Projects/rag-training/data/legal/sla_contract.txt:0:0-0:0), [.csv](cci:7://file:///c:/Users/jan.petr/OneDrive%20-%20dolphinconsulting.cz/Projects/rag-training/data/finance/financial_report.csv:0:0-0:0) nebo [.md](cci:7://file:///c:/Users/jan.petr/OneDrive%20-%20dolphinconsulting.cz/Projects/rag-training/data/tech_docs/api_docs.md:0:0-0:0) – naším cílem je převést vše na jednotný objekt **Dokument**, se kterým umí RAG systém pracovat.

#### 1. Co je to `Document` objekt?
Každý načtený kus dat je v LangChainu reprezentován objektem, který má dvě hlavní části:
* **`page_content`**: Samotný text (obsah).
* **`metadata`**: Informace o původu (např. `{source: "cesta/k/souboru.txt", row: 5}`).

#### 2. Jak fungují jednotlivé Loadery?
Loadery se starají o převod surových dat na text.
* **`TextLoader`**: "Hloupý" loader. Vezme celý obsah souboru tak, jak je, a vloží ho do jednoho dokumentu.
* **`CSVLoader`**: "Chytrý" loader. Každý řádek tabulky převede na **samostatný dokument**. Data zformátuje do dvojic `Sloupec: Hodnota`, aby si LLM zachovalo kontext (ví, co které číslo znamená).
* **`UnstructuredMarkdownLoader`**: Snaží se parsovat strukturu. Často odstraňuje formátovací znaky (jako `**` pro tučné písmo), aby zbyl čistý kontextuální text.

#### 3. Co se děje se znaky? (Encoding & Normalizace)
* **Kódování**: Systém očekává standardní **UTF-8**. Pokud jsou v souboru české znaky uložené v jiném kódování (např. Windows-1250), loader může selhat nebo zobrazit nesmysly.
* **Opravy chyb**: Loadery **nefungují** jako spell-checker. Pokud je ve zdrojovém textu překlep, bude i v načteném dokumentu.
* **Formátování**: Bílé znaky (mezery, konce řádků) jsou často sjednoceny (normalizovány), aby byl text kompaktnější.

In [3]:
from langchain_community.document_loaders import TextLoader, JSONLoader, CSVLoader, UnstructuredMarkdownLoader

ts_path = "../data/legal/sla_contract.txt"
md_path = "../data/tech_docs/api_docs.md"
csv_path = "../data/finance/financial_report.csv"

# 1. Text Loader
text_loader = TextLoader(ts_path)
documents_text = text_loader.load()

# 2. Markdown Loader
md_loader = UnstructuredMarkdownLoader(md_path)
documents_md = md_loader.load()

# 3. CSV Loader
csv_loader = CSVLoader(csv_path)
documents_csv = csv_loader.load()

print(f"Loaded {len(documents_text)} text docs, {len(documents_md)} md docs, {len(documents_csv)} csv rows.")

Loaded 1 text docs, 1 md docs, 4 csv rows.


In [4]:
# 1. Zobrazení obsahu textového dokumentu
print("--- TEXT DOCUMENT ---")
print(documents_text[0].page_content)
print("Metadata:", documents_text[0].metadata)

# 2. Zobrazení obsahu Markdown dokumentu
print("\n--- MARKDOWN DOCUMENT ---")
# Zobrazíme třeba jen prvních 500 znaků, pokud je to dlouhé
print(documents_md[0].page_content[:500]) 

# 3. Zobrazení CSV (každý řádek je obvykle jeden dokument)
print("\n--- CSV ROW 1 ---")
print(documents_csv[0].page_content)
print("Metadata:", documents_csv[0].metadata)

--- TEXT DOCUMENT ---
SERVICE LEVEL AGREEMENT (SLA) for NebulaDB Enterprise

1. DEFINITIONS
"Uptime" refers to the availability of the Service during a billing month.
"Downtime" refers to a period of time exceeding 5 minutes where the Error Rate is greater than 5%.

2. SERVICE COMMITMENT
NebulaDB commits to a Monthly Uptime Percentage of at least 99.99% ("Service Commitment").

3. SERVICE CREDITS
If we do not meet the Service Commitment, you will be eligible to receive a Service Credit as follows:
- < 99.99% but >= 99.0%: 10% Credit
- < 99.0% but >= 95.0%: 25% Credit
- < 95.0%: 100% Credit

4. EXCLUSIONS
The Service Commitment does not apply to any unavailability, suspension, or termination of NebulaDB performance issues: (i) caused by factors outside of our reasonable control, including any force majeure event or Internet access or related problems beyond the demarcation point of NebulaDB.

5. CLAIMS
To receive a Service Credit, you must submit a claim by opening a case in the NebulaD

## 3. Chunking (Text Splitting)

**Concept:** LLMs have context windows. We can't stuff a 100-page contract into one prompt. Also, retrieving smaller, specific chunks is more accurate than retrieving whole documents.

**Technique:**
- `CharacterTextSplitter`: Simple, splits by character count.
- `RecursiveCharacterTextSplitter`: Smarter, tries to keep paragraphs and sentences together.

# 🔧 Detailní vysvětlení parametrů `RecursiveCharacterTextSplitter`

## 1. Zásadní parametry (Ovlivňují kvalitu RAG)

### `chunk_size` (int)
*   **Default:** `4000`
*   **Co to je:** Maximální cílová velikost jednoho kousku textu (chunku).
*   **Vliv:** Pokud je příliš velký, LLM se zahltí balastem. Pokud příliš malý, ztratíte kontext.
*   **Doporučení:** Pro GPT-4 a RAG se obvykle nastavuje na **500 až 1000**.

### `chunk_overlap` (int)
*   **Default:** `200`
*   **Co to je:** Počet znaků, které se překrývají mezi koncem jednoho a začátkem druhého chunku.
*   **Vliv:** Zabraňuje tomu, aby důležitá informace (např. věta) byla "rozříznuta" v půlce a ztratila smysl.
*   **Doporučení:** Obvykle **10–20 %** velikosti `chunk_size`.

### `separators` (list[str] | None)
*   **Default:** `None` (Interně: `["\n\n", "\n", " ", ""]`)
*   **Co to je:** Seznam oddělovačů seřazený podle priority (od nejsilnějšího po nejslabší).
*   **Vliv:** Určuje strategii dělení. Splitter zkouší dělit podle 1. oddělovače (odstavce). Pokud je chunk stále moc velký, zkusí 2. oddělovač (řádky), pak 3. (slova) atd.
*   **Kdy měnit:** Pokud dělíte kód (použijte `RecursiveCharacterTextSplitter.from_language`) nebo specifická data.

---

## 2. Parametry chování oddělovačů

### `keep_separator` (bool | "start" | "end")
*   **Default:** `True`
*   **Co to je:** Určuje, co se stane se znakem, podle kterého se dělilo (např. `\n\n`).
    *   `True` (nebo `"end"`): Oddělovač zůstane na **konci** předchozího chunku.
    *   `"start"`: Oddělovač se přesune na **začátek** následujícího chunku.
    *   `False`: Oddělovač se úplně vymaže.
*   **Vliv:** Zachování (`True`) pomáhá LLM pochopit strukturu textu (že tam byl konec odstavce).

### `is_separator_regex` (bool)
*   **Default:** `False`
*   **Co to je:** Pokud `True`, chápe položky v `separators` jako regulární výrazy (RegEx) místo obyčejného textu.
*   **Vliv:** Umožňuje složitější pravidla dělení (např. "rozděl v místě, kde je tečka následovaná velkým písmenem").

### `strip_whitespace` (bool)
*   **Default:** `True`
*   **Co to je:** Pokud `True`, odstraní nadbytečné mezery a prázdné znaky na úplném začátku a konci hotového chunku.
*   **Vliv:** Čistí data, aby chunks nezačínaly zbytečnými entery nebo mezerami.

---

## 3. Technické a mapovací parametry

### `length_function` (Callable)
*   **Default:** `len` (standardní funkce Pythonu pro počet znaků)
*   **Co to je:** Metr, kterým splitter měří délku textu. Určuje, co vlastně znamená číslo v `chunk_size`.
*   **Jak to funguje:**
    *   **Znaky (Default):** Pokud parametr nezměníte, splitter používá funkci `len()`. Nastavení `chunk_size=500` pak znamená **maximálně 500 znaků** (písmen, mezer, interpunkce). Je to rychlé, ale pro LLM méně přesné (model "nevidí" znaky, ale tokeny).
    *   **Tokeny (Pokročilé):** Pro RAG je často lepší měřit přímo v jednotkách, kterým rozumí model (tokeny). Nastavení `chunk_size=500` pak znamená **maximálně 500 tokenů**. To zajistí, že efektivněji využijete kontextové okno modelu.
*   **Příklad nastavení na tokeny:**
    Pokud chcete dělit podle tokenů (např. pro modely OpenAI), musíte použít knihovnu `tiktoken`:

    ```python
    import tiktoken

    # Vytvoříme funkci, která počítá tokeny místo znaků
    def tiktoken_len(text):
        tokenizer = tiktoken.get_encoding("cl100k_base") # encoding pro GPT-3.5/4
        tokens = tokenizer.encode(text)
        return len(tokens)

    # Předáme tuto funkci splitteru
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,  # Nyní to znamená 500 TOKENŮ
        chunk_overlap=50,
        length_function=tiktoken_len
    )
    ```
### `add_start_index` (bool)
*   **Default:** `False`
*   **Co to je:** Pokud `True`, přidá do metadat chunku klíč `start_index` s číslem, na kterém znaku v původním souboru tento chunk začíná.
*   **Vliv:** Užitečné pro zvýrazňování (highlighting) nalezeného textu v původním dokumentu v UI aplikace.

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
all_docs = documents_text + documents_md + documents_csv
chunks = splitter.split_documents(all_docs)
print(f"Split {len(all_docs)} documents into {len(chunks)} chunks.")
if len(chunks) > 0:
    print("Example chunk:", chunks[0].page_content)

Split 6 documents into 9 chunks.
Example chunk: SERVICE LEVEL AGREEMENT (SLA) for NebulaDB Enterprise

1. DEFINITIONS
"Uptime" refers to the availability of the Service during a billing month.
"Downtime" refers to a period of time exceeding 5 minutes where the Error Rate is greater than 5%.

2. SERVICE COMMITMENT
NebulaDB commits to a Monthly Uptime Percentage of at least 99.99% ("Service Commitment").


In [6]:
# --- INSPEKCE PROMĚNNÝCH (VARIABLE INSPECTION) ---

# 1. Analýza `all_docs` (Všechny dokumenty před rozdělením)
# Co to je: Seznam všech dokumentů načtených loadery. Každý dokument je objekt 'Document'.
print(f"🔍 Původní dokumenty (all_docs): {len(all_docs)} ks")
try:
    print("   - Typ objektu:", type(all_docs[0]))
    # Zobrazíme detail prvního dokumentu (např. SLA kontrakt)
    if len(all_docs) > 0:
        print(f"   - Ukázka 1. dokumentu (znaků: {len(all_docs[0].page_content)}):")
        print(f"     '{all_docs[0].page_content[:100]}...'") # Jen prvních 100 znaků
        print(f"     Metadata: {all_docs[0].metadata}")
except Exception as e:
    print(f"   - Chyba při zobrazování all_docs: {e}")

print("\n" + "="*40 + "\n")

# 2. Analýza `chunks` (Rozsekané kousky)
# Co to je: Seznam dokumentů PO rozdělení splitterem.
# Proč: Protože původní dokumenty mohou být pro LLM příliš dlouhé a nevešly by se do kontextu.
print(f"🔪 Rozsekané části (chunks): {len(chunks)} ks")
try:
    if len(chunks) > 0:
        print("   - Typ objektu:", type(chunks[0]), "(Stále je to Document!)")

        # Porovnání: Kolik chunků vzniklo z prvního dokumentu?
        # Filtrujeme chunky, které mají stejný 'source' jako první dokument
        if len(all_docs) > 0:
            source_file = all_docs[0].metadata.get('source')
            matching_chunks = [c for c in chunks if c.metadata.get('source') == source_file]

            print(f"   - Původní soubor '{source_file}' byl rozdělen na {len(matching_chunks)} částí.")
            if len(matching_chunks) > 0:
                print("   - Ukázka 1. chunku:")
                print(f"     '{matching_chunks[0].page_content}'")
                print(f"     Metadata chunku: {matching_chunks[0].metadata}")
except Exception as e:
    print(f"   - Chyba při zobrazování chunks: {e}")

🔍 Původní dokumenty (all_docs): 6 ks
   - Typ objektu: <class 'langchain_core.documents.base.Document'>
   - Ukázka 1. dokumentu (znaků: 1027):
     'SERVICE LEVEL AGREEMENT (SLA) for NebulaDB Enterprise

1. DEFINITIONS
"Uptime" refers to the availab...'
     Metadata: {'source': '../data/legal/sla_contract.txt'}


🔪 Rozsekané části (chunks): 9 ks
   - Typ objektu: <class 'langchain_core.documents.base.Document'> (Stále je to Document!)
   - Původní soubor '../data/legal/sla_contract.txt' byl rozdělen na 3 částí.
   - Ukázka 1. chunku:
     'SERVICE LEVEL AGREEMENT (SLA) for NebulaDB Enterprise

1. DEFINITIONS
"Uptime" refers to the availability of the Service during a billing month.
"Downtime" refers to a period of time exceeding 5 minutes where the Error Rate is greater than 5%.

2. SERVICE COMMITMENT
NebulaDB commits to a Monthly Uptime Percentage of at least 99.99% ("Service Commitment").'
     Metadata chunku: {'source': '../data/legal/sla_contract.txt'}


In [12]:
all_docs

[Document(metadata={'source': '../data/legal/sla_contract.txt'}, page_content='SERVICE LEVEL AGREEMENT (SLA) for NebulaDB Enterprise\n\n1. DEFINITIONS\n"Uptime" refers to the availability of the Service during a billing month.\n"Downtime" refers to a period of time exceeding 5 minutes where the Error Rate is greater than 5%.\n\n2. SERVICE COMMITMENT\nNebulaDB commits to a Monthly Uptime Percentage of at least 99.99% ("Service Commitment").\n\n3. SERVICE CREDITS\nIf we do not meet the Service Commitment, you will be eligible to receive a Service Credit as follows:\n- < 99.99% but >= 99.0%: 10% Credit\n- < 99.0% but >= 95.0%: 25% Credit\n- < 95.0%: 100% Credit\n\n4. EXCLUSIONS\nThe Service Commitment does not apply to any unavailability, suspension, or termination of NebulaDB performance issues: (i) caused by factors outside of our reasonable control, including any force majeure event or Internet access or related problems beyond the demarcation point of NebulaDB.\n\n5. CLAIMS\nTo receiv

## 4. Vector Stores & Embeddings

**Concept:** Computers don't understand text meaning, they understand numbers. **Embeddings** convert text into a vector (a list of numbers) where similar meanings are close together in space.

We store these vectors in a **Vector Store** (like ChromaDB) to perform semantic search.

# 🧠 Co se děje v části "Embeddings & Vector Store"?

V této fázi měníme **lidskou řeč** na **řeč strojovou**, abychom v ní mohli efektivně vyhledávat.

## 1. Embedding Model (Překladač)
*   **Co to je:** Služba (v našem případě `text-embedding-ada-002` na Azure), která vezme jakýkoli text a vrátí dlouhý seznam čísel (vektor).
*   **Jak to funguje:** Model chápe významy slov. Slova s podobným významem (např. *král* a *císař*) budou mít čísla velmi blízko sebe.
*   **Výsledek:** Každý náš "chunk" textu se promění na seznam 1536 čísel.

## 2. Vector Store (Databáze významů)
*   **Co to je:** Specializovaná databáze (zde `ChromaDB`), která neumí jen ukládat text, ale hlavně ty číselné vektory.
*   **Proč to potřebujeme:** Klasická databáze hledá přesnou shodu (*obsahuje slovo "smlouva"?*). Vektorová databáze hledá **významovou blízkost** (*je tento dotaz podobný obsahu smlouvy?*).
*   **Proces:**
    1.  Vezmeme všechny `chunks`.
    2.  Pošleme je do Azure na "přeložení" (embed).
    3.  Vrátí se nám vektory.
    4.  Uložíme je do `vectorstore` spolu s původním textem a metadaty.

In [7]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os, shutil

# POKUD MÁTE .ENV SOUBOR, HODNOTY SE NAČTOU Z NĚJ.
# JINAK JE DOPLŇTE PŘÍMO DO UVOZOVEK:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "https://VAS_RESOURCE_NAME.openai.azure.com/")
api_key = os.getenv("AZURE_OPENAI_API_KEY", "VAS_API_KEY") # V prostředí často jako AZURE_OPENAI_API_KEY
deployment_name = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-ada-002") # Název nasazení modelu (ne modelu samotného!)
api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2023-05-15") 

# Inicializace Azure Embeddings
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    azure_deployment=deployment_name,
    openai_api_version=api_version,
)


import stat  # Potřeba importovat

# Tahle pomocná funkce "odpojí" read-only zámek, pokud na něj narazí
def force_remove_readonly(func, path, exc_info):
    os.chmod(path, stat.S_IWRITE) # Změníme práva na zápis
    func(path) # Zkusíme operaci smazání znovu

persist_directory = "../chroma_db"
if os.path.exists(persist_directory):
    # Přidáme parametr onerror, který zavolá naši funkci při chybě
    shutil.rmtree(persist_directory, onerror=force_remove_readonly)
    print(f"🗑️ Smazána stará databáze: {persist_directory}")

# 2. Vytvoření nové databáze s persistencí
# Nyní vytvoříme novou databázi a rovnou ji uložíme na disk.
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="rag_training_v1",
    persist_directory=persist_directory
)
print(f"✅ Nový VectorStore vytvořen a uložen do: {persist_directory}")

print("VectorStore created with Azure OpenAI!")

🗑️ Smazána stará databáze: ../chroma_db
✅ Nový VectorStore vytvořen a uložen do: ../chroma_db
VectorStore created with Azure OpenAI!


# 🧠 Jak fungují Embeddings a Vector Store? (Vysvětlení pro lidi)

Představte si celý proces jako **vztah mezi Kuchařkou (Vector Store) a Chuťovými buňkami (Embedding Model)**. Zde je vysvětlení krok za krokem.

## 1. Co je to vlastně "Embedding"?
**Embedding je překladač z lidštiny do "číselné řeči", které rozumí počítač.**

Představte si, že máte mapu světa.
*   Když řeknu **"Pes"**, embedding model převede toto slovo na přesné souřadnice: `[50.123, 14.456]`.
*   Když řeknu **"Kočka"**, model řekne: `[50.125, 14.458]` (leží to na mapě kousek vedle psa).
*   Když řeknu **"Vesmírná raketa"**, model řekne: `[-20.000, 80.555]` (úplně jiný světadíl).

To dlouhé pole čísel (v kódu proměnná `vector`), které vidíte ve výstupu, jsou přesně tyto "GPS souřadnice významu" v obrovském, 1536-rozměrném prostoru.

---

## 2. Musí mít embedding "trvalý charakter"?
**Rozhodně ANO. To je naprosto klíčové.**

Musíte vždy používat **ten samý model** (stejný klíč k mapě). Představte si dva různé modely jako dvě různé mapy:
1.  **Model A (OpenAI):** Mapa Prahy.
2.  **Model B (HuggingFace):** Mapa New Yorku.

Pokud byste dokumenty uložili pomocí **Modelu A** (souřadnice v Praze) a pak se ptali pomocí **Modelu B** (hledáte souřadnice v New Yorku), nikdy se nepotkáte. Proto v celém notebooku používáme jednu proměnnou `embeddings`.

---

## 3. Jak se "Testovací slovo" a "Vector Store" propojí?
Toto je ten magický moment RAGu (Retrieval-Augmented Generation). Funguje to ve třech krocích:

### Krok A: Uložení (Ingestion) – "Zapichování vlaječek"
1.  Vezmeme větu ze smlouvy: *"Smlouva platí 5 let."*
2.  Pošleme ji do `embeddings` modelu.
3.  Model vrátí souřadnice: `[0.1, 0.5, 0.9 ...]`.
4.  Tyto souřadnice **uložíme do Vector Store** (ChromaDB) a dáme k nim lísteček s původním textem.
    *   *Efekt: Máme databázi plnou vlaječek zapíchaných na mapě podle významu.*

### Krok B: Dotaz (Query) – "Kde jsem já?"
1.  Uživatel (nebo vy v testu) napíše dotaz: *"Jak dlouho to platí?"* (Všimněte si, že jsme nepoužili stejná slova).
2.  Tento dotaz pošleme do **toho samého `embeddings` modelu**.
3.  Model vrátí souřadnice pro tento dotaz: `[0.11, 0.51, 0.89 ...]`.

### Krok C: Hledání (Search) – "Kdo je mým sousedem?"
1.  Vektorová databáze dostane souřadnice vašeho dotazu.
2.  Rozhlédne se kolem tohoto bodu a hledá **nejbližší zapíchnutou vlaječku**.
3.  Najde vlaječku *"Smlouva platí 5 let"*, protože její souřadnice jsou matematicky nejblíž (leží na stejném "náměstí významu").
4.  Vrátí vám ten původní text.

In [8]:
# --- INSPEKCE EMBEDDINGS A VECTOR STORE ---

# 1. Testovací Embedding
# Co to je: Vektor, který reprezentuje význam slova "test".
# Proč: Ověříme, že model funguje a podíváme se, jak vypadá "strojová řeč".
test_word = "apple"
try:
    vector = embeddings.embed_query(test_word)
    print(f"📊 Testovací slovo: '{test_word}'")
    print(f"   - Délka vektoru: {len(vector)} dimenzí (standard pro model ada-002 je 1536)")
    print(f"   - Prvních 5 čísel vektoru: {vector[:5]}...")
    print(f"   - Typ dat: {type(vector)}")
except Exception as e:
    print(f"❌ Chyba při vytváření embeddingu: {e}")

print("\n" + "="*40 + "\n")

# 2. Vector Store (ChromaDB)
# Co to je: Databáze, kde jsou uloženy vektory všech našich chunků.
# Proč: Abychom v nich mohli rychle hledat podle podobnosti.
print(f"🗄️ VectorStore (ChromaDB):")
# ChromaDB v LangChainu zapouzdřuje kolekci
try:
    if hasattr(vectorstore, "_collection"):
        print(f"   - Jméno kolekce: {vectorstore._collection.name}")
        print(f"   - Počet uložených vektorů: {vectorstore._collection.count()}")

    # Zkusíme najít něco podobného slovu "contract"
    # Toto ověří, že vyhledávání funguje end-to-end
    results = vectorstore.similarity_search("contract", k=1)
    if results:
        print(f"\n🔎 Rychlý test hledání slova 'contract':")
        print(f"   - Nalezený dokument: '{results[0].page_content[:100]}...'")
        print(f"   - Zdroj: {results[0].metadata}")
    else:
        print("\n⚠️ Hledání vrátilo 0 výsledků (to je divné, pokud máme data).")

except Exception as e:
    print(f"❌ Chyba při práci s VectorStore: {e}")

📊 Testovací slovo: 'apple'
   - Délka vektoru: 1536 dimenzí (standard pro model ada-002 je 1536)
   - Prvních 5 čísel vektoru: [0.007782285567373037, -0.023080386221408844, -0.007522648200392723, -0.02772652730345726, -0.00455048494040966]...
   - Typ dat: <class 'list'>


🗄️ VectorStore (ChromaDB):
   - Jméno kolekce: rag_training_v1
   - Počet uložených vektorů: 9

🔎 Rychlý test hledání slova 'contract':
   - Nalezený dokument: 'Quarter: Q4 2023
Revenue: 950000
Expenses: 750000
Profit: 200000
Notes: Previous year comparison....'
   - Zdroj: {'source': '../data/finance/financial_report.csv', 'row': 3}


## 5. Basic Retrieval (Sémantické vyhledávání)

V předchozím kroku jsme dokumenty rozsekali, převedli na čísla (vektory) a uložili do **Vektorové databáze (Chroma)**. Nyní si ukážeme, jak v těchto datech hledat odpovědi.

### ⚙️ Jak to funguje pod kapotou?

Kód `docs = vectorstore.similarity_search(question, k=3)` vypadá jednoduše, ale spouští komplexní proces, který spojuje vše, co jsme doteď udělali.

Aby databáze mohla najít odpověď, musí se stát následující:

1.  **Transformace otázky (Embeddings):**
    *   Váš dotaz (`question`) je pouze text. Databáze ale rozumí jen číslům.
    *   Proto se systém (objekt `vectorstore`) podívá na svou konfiguraci **Embeddings** (náš "model/překladač").
    *   Pošle váš dotaz do modelu (Azure OpenAI) a získá zpět **vektor otázky** (seznam čísel reprezentující význam otázky).
    *   **Důležité:** Musíme použít *úplně stejný* model (Embeddings), jaký jsme použili při ukládání dokumentů. Jinak by čísla neseděla.

2.  **Matematické porovnání (Similarity Search):**
    *   ChromaDB vezme **vektor otázky** a porovná ho se všemi miliardami **vektorů dokumentů**, které má uložené.
    *   Hledá tzv. *nejbližší sousedy* – tedy dokumenty, jejichž významový vektor je matematicky nejblíže vektoru otázky.

3.  **Výběr kontextu (Retrieval):**
    *   Databáze vrátí `k` (např. 3) nejlepších úryvků textu. Tyto úryvky pak předhodíme LLM jako podklad pro odpověď.

---

> 💡 **Architektura v reálné aplikaci vs. Notebook**
>
> V tomto výukovém notebooku děláme vše najednou: vytvoříme databázi a hned se jí zeptáme. V reálné produkční aplikaci ("chatbotovi") jsou tyto procesy oddělené:
>
> 1.  **Ingestion Pipeline (Noční proces):** Skript, který jednou za čas vezme dokumenty, vytvoří/aktualizuje databázi a **uloží ji na disk**.
> 2.  **Aplikace (Chatbot):** Běží neustále. Při startu nic nevytváří, pouze se **připojí** k již existující databázi na disku (tzv. načte `persisted_directory` a použije stejný `embeddings` model) a rovnou hledá.

In [19]:
question = "What is the SLA for downtimes?"



# --- SIMULACE: Načtení již existující databáze (Query Only) ---
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
# 1. Musíme inicializovat Embeddings
# Databáze potřebuje vědět, jakým "jazykem" (modelem) má převádět váš dotaz na čísla.
# Musí to být STEJNÝ model, jakým jste data uložili!
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-ada-002"),
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2023-05-15"),
)
# 2. Načtení existujícího VectorStore z disku
# Nepoužíváme .from_documents(), protože nechceme nic vkládat.
# Jen řekneme: "Tady je složka s daty, použij ji."
persist_directory = "../chroma_db"  # Cesta ke složce z předchozích kroků
if os.path.exists(persist_directory):
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings,
        collection_name="rag_training_v1"
    )
    print(f"✅ Úspěšně připojeno k databázi. Obsahuje {vectorstore._collection.count()} dokumentů.")
else:
    print(f"❌ Chyba: Složka {persist_directory} neexistuje. Musíte nejdříve spustit Ingestion část (vytvoření databáze).")





docs = vectorstore.similarity_search(question, k=3)

for i, doc in enumerate(docs):
    print(f"\n[Result {i+1}] Source: {doc.metadata.get('source')}")
    print(doc.page_content)

C:\Users\jan.petr\AppData\Local\Temp\ipykernel_57296\3287707502.py:23: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


✅ Úspěšně připojeno k databázi. Obsahuje 9 dokumentů.

[Result 1] Source: ../data/legal/sla_contract.txt
SERVICE LEVEL AGREEMENT (SLA) for NebulaDB Enterprise

1. DEFINITIONS
"Uptime" refers to the availability of the Service during a billing month.
"Downtime" refers to a period of time exceeding 5 minutes where the Error Rate is greater than 5%.

2. SERVICE COMMITMENT
NebulaDB commits to a Monthly Uptime Percentage of at least 99.99% ("Service Commitment").

[Result 2] Source: ../data/legal/sla_contract.txt
4. EXCLUSIONS
The Service Commitment does not apply to any unavailability, suspension, or termination of NebulaDB performance issues: (i) caused by factors outside of our reasonable control, including any force majeure event or Internet access or related problems beyond the demarcation point of NebulaDB.

5. CLAIMS
To receive a Service Credit, you must submit a claim by opening a case in the NebulaDB Support Center within 30 days of the incident.

[Result 3] Source: ../data/legal/s

## 6. Generation (The 'G' in RAG)

Finally, we pass the retrieved chunks to the LLM to write the answer.

# 🛠️ Anatomie RAG řetězce: Co se děje pod kapotou?
Tento dokument vysvětluje logiku RAG (Retrieval-Augmented Generation) pipeline postavené pomocí moderního **LCEL (LangChain Expression Language)**. Jednotlivé kroky na sebe navazují jako na výrobní lince.
## 1. Definice LLM (Mozek)
```python
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",  # Váš deployment v Azure
    openai_api_version="2023-05-15",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    temperature=0
)
```
*   **Co se děje:** Inicializujeme spojení s jazykovým modelem (GPT-4o) běžícím v cloudu Azure.
*   **Proč to děláme:** Potřebujeme "mozek", který na konci procesu vygeneruje srozumitelnou odpověď na základě nalezených dat.
*   **Klíčový parametr:** `temperature=0`. Pro RAG systémy chceme nulu, aby model "nehalucinoval" a držel se striktně nalezených faktů.
## 2. Retriever (Vyhledávač)
```python
retriever = vectorstore.as_retriever()
```

*   **Co se děje:** Převádíme statickou databázi (VectorStore) na aktivní vyhledávací komponentu.
*   **Proč to děláme:** VectorStore umí jen ukládat. My potřebujeme funkci, do které hodíme otázku (string) a ona nám vrátí seznam relevantních dokumentů.
## 3. Prompt Template (Šablona instrukcí)
```python
template = """Answer the question based only on the following context: {context}

Question: {question} """

prompt = ChatPromptTemplate.from_template(template)
```

*   **Co se děje:** Připravujeme "formu", do které se budou vkládat data.
*   **Logika:** Šablona má dvě místa (proměnné), které je třeba vyplnit:
    1.  `{context}`: Sem se vloží texty nalezené v dokumentech (vaše znalostní báze).
    2.  `{question}`: Sem se vloží aktuální dotaz uživatele.

## 4. Helper Funkce (Formátovač dokumentů)
```python
def format_docs(docs): 
    return "\n\n".join([d.page_content for d in docs])
```
*   **Co se děje:** Tato funkce bere složité objekty `Document` (které vrací retriever) a vytahuje z nich čistý text.
*   **Proč to děláme:** Retriever vrací metadata, zdrojové cesty atd. LLM ale zajímá jen obsah. Funkce vezme obsah všech nalezených dokumentů a spojí je do jednoho dlouhého textu odděleného mezerami, aby se vešel do proměnné `{context}`.

## 5. Sestavení Řetězce (The Pipeline)
Toto je nejdůležitější část. Definujeme tok dat pomocí operátoru `|` (pipe), který posílá výstup z jedné funkce jako vstup do další.
```python
rag_chain = ( {"context": retriever | format_docs, "question": RunnablePassthrough()}
 | prompt
 | llm
 | StrOutputParser() )


 # Pseudo-kód toho, co se děje uvnitř jen pro ukázku:
rag_chain = RunnableSequence(
    first=RunnableParallel(  # Ten slovník na začátku
        context=(retriever | format_docs),
        question=RunnablePassthrough()
    ),
    middle=[
        prompt,  # ChatPromptTemplate
        llm,     # AzureChatOpenAI
        StrOutputParser()
    ]
)
```
### Krok A: Paralelní příprava dat (Slovník na začátku)
`{"context": ..., "question": ...}`
Tento blok běží jako první a připravuje data pro prompt. Dělá dvě věci naráz:
1.  **Získání kontextu:** `retriever | format_docs`
    *   Vezme vstupní otázku -> pošle ji do `retriever` (najde dokumenty) -> výsledek pošle do `format_docs` (převede na text) -> uloží jako `context`.
2.  **Předání otázky:** `"question": RunnablePassthrough()`
    *   `RunnablePassthrough()` je "průtokový ohřívač". Vezme vstupní otázku a beze změny ji pošle dál pod klíčem `question`.
### Krok B: Vytvoření zadání
`| prompt`
*   Vezme data z kroku A (naplněný `context` i `question`) a vloží je do šablony z bodu 3. Výsledkem je finální text zadání pro model.
### Krok C: Generování odpovědi
`| llm`
*   Hotové zadání se pošle do GPT-4o. Model vygeneruje odpověď (objekt typu `AIMessage`).
### D. Čistý výstup
`| StrOutputParser()`
*   Model vrací složitý objekt. Tento parser z něj vytáhne čistý řetězec (string) s odpovědí, kterou vidí uživatel.
## 6. Spuštění (Execution)
response = rag_chain.invoke(question)

*   **Co se děje:** Metoda `.invoke()` je spouštěcí tlačítko.
*   Funkce vezme proměnnou `question` (váš dotaz) a hodí ji na začátek řetězce (do bodu 5).
*   Celý proces proběhne automaticky a na konci vypadne textová odpověď.

In [12]:
# Moderní a spolehlivá alternativa k RetrievalQA
from langchain_openai import AzureChatOpenAI
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. Definice LLM (Používáme Azure variantu)
# Načteme proměnné prostředí, které jste definoval v předchozích buňkách nebo .env souboru
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",  # Předpokládáme, že název deploymentu v Azure je "gpt-4o"
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2023-05-15"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    temperature=0
)

# 2. Retriever (Vyhledávač)
retriever = vectorstore.as_retriever()

# 3. Prompt (Instrukce pro model)
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 4. Pomocná funkce pro zformátování dokumentů do textu
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# 5. Sestavení RAG řetězce (Chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 6. Spuštění
# .invoke() spustí celý proces: najde dokumenty -> vloží do promptu -> odešle na GPT-4
print(f"Otázka: {question}")
print("-" * 30)
response = rag_chain.invoke(question)
print("Odpověď:", response)

Otázka: What is the SLA for downtimes?
------------------------------
Odpověď: The SLA for downtimes in the NebulaDB Enterprise Service Level Agreement specifies that "Downtime" refers to a period of time exceeding 5 minutes where the Error Rate is greater than 5%.


In [18]:
print(type(rag_chain))

<class 'langchain_core.runnables.base.RunnableSequence'>
